In [1]:
import numpy as np

In [2]:
#oppacket = "C200B40A82" #3
#oppacket = "04005AC33890" #54
#oppacket = "880086C3E88112" #7
#oppacket = "CE00C43D881120" #9
#oppacket = "D8005AC2A8F0" #1
#oppacket = "F600BC2D8F" #0
#oppacket = "9C005AC2F8F0" #0
#oppacket = "9C0141080250320F1802104A08" #1

In [3]:
oppacket = "220D6448300428021F9EFE668D3F5FD6025165C00C602FC980B45002A40400B402548808A310028400C001B5CC00B10029C0096011C0003C55003C0028270025400C1002E4F19099F7600142C801098CD0761290021B19627C1D3007E33C4A8A640143CE85CB9D49144C134927100823275CC28D9C01234BD21F8144A6F90D1B2804F39B972B13D9D60939384FE29BA3B8803535E8DF04F33BC4AFCAFC9E4EE32600C4E2F4896CE079802D4012148DF5ACB9C8DF5ACB9CD821007874014B4ECE1A8FEF9D1BCC72A293A0E801C7C9CA36A5A9D6396F8FCC52D18E91E77DD9EB16649AA9EC9DA4F4600ACE7F90DFA30BA160066A200FC448EB05C401B8291F22A2002051D247856600949C3C73A009C8F0CA7FBCCF77F88B0000B905A3C1802B3F7990E8029375AC7DDE2DCA20C2C1004E4BE9F392D0E90073D31634C0090667FF8D9E667FF8D9F0C01693F8FE8024000844688FF0900010D8EB0923A9802903F80357100663DC2987C0008744F8B5138803739EB67223C00E4CC74BA46B0AD42C001DE8392C0B0DE4E8F660095006AA200EC198671A00010E87F08E184FCD7840289C1995749197295AC265B2BFC76811381880193C8EE36C324F95CA69C26D92364B66779D63EA071008C360098002191A637C7310062224108C3263A600A49334C19100A1A000864728BF0980010E8571EE188803D19A294477008A595A53BC841526BE313D6F88CE7E16A7AC60401A9E80273728D2CC53728D2CCD2AA2600A466A007CE680E5E79EFEB07360041A6B20D0F4C021982C966D9810993B9E9F3B1C7970C00B9577300526F52FCAB3DF87EC01296AFBC1F3BC9A6200109309240156CC41B38015796EABCB7540804B7C00B926BD6AC36B1338C4717E7D7A76378C85D8043F947C966593FD2BBBCB27710E57FDF6A686E00EC229B4C9247300528029393EC3BAA32C9F61DD51925AD9AB2B001F72B2EE464C0139580D680232FA129668"

In [4]:
oppacket_bin = ""
for c in oppacket:
    v = int(c, base=16)
    binv = format(v,"04b") #transvorms value to binary with exactly 4 characters
    oppacket_bin += binv
#oppacket_bin

In [11]:
def readPacket(s):
    #Get first 3 bits for packet version
    version = int(s[:3], base=2)
    packid = int(s[3:6],base=2)
    print(f"version: {version} , packid: {packid}")

    remaining_str = None
    packvalue = None
    
    if packid==4:
        print("pack is a literal value")
        hasliterals=True
        lit_value_str=""
        i=6
        while hasliterals:
            lit_s = s[i:i+5]
            if lit_s[0]=="0":
                hasliterals=False
            #Read the actual value
            lit_value_str += lit_s[1:]
            i=i+5
        print(f"lit_value_str = {lit_value_str}")
        lit_value_int = int(lit_value_str, base=2)
        print(f"lit_value_int={lit_value_int}")
        remaining_str = s[i:]
        
        packvalue= lit_value_int
        #return {"value": lit_value_int, "remainstr": remaining_str}
    else:
        #Operation
        length_type_id = s[6]
        print(f"length type ID = {length_type_id}")
        subpack_values=[]
        if length_type_id=="0":
            #Next 15 bits are the total length of the sub-packets
            print(f"15 bit string: {s[7:22]}")
            subpacks_total_length = int(s[7:22], base=2)
            print(f"subpacks_total_length={subpacks_total_length}")

            subpacks = s[22:22+subpacks_total_length]
            print(f"subpacks : {subpacks}")
            #Handle the subpacks
            hasSubpacks=True
            ipack=0
            while hasSubpacks:
                print(f"ipack= {ipack}")
                if subpacks != '':
                    ret = readPacket(subpacks)
                    subpackvalue = ret["value"]
                    subpack_values.append(subpackvalue)
                    
                    subpacks= ret["remainstr"]
                    if subpacks == '' or subpacks is None:
                        hasSubpacks=False
                else:
                    hasSubpacks=False
                ipack+=1
                
            remaining_str = s[22+subpacks_total_length:]

        else:
            assert length_type_id == "1", "Error in length_type_id"
            #Next 11 bits are the number of subpackets immediately after
            print(s[7:18])
            n_subpackets =int(s[7:18], base=2)
            print(f"n_subpackets={n_subpackets}")

            remaining_str = s[18:]
            for i in range(n_subpackets):
                print(f"packet i={i}")
                ret = readPacket(remaining_str)
                assert "remainstr" in ret, "ERROR: no remainstr"
                remaining_str= ret["remainstr"]

                subpackvalue = ret["value"]
                subpack_values.append(subpackvalue)

                i+=1
            #return {"value":0 , "remainstr": remaining_str}

        #Apply the operation to the values collected
        values_np = np.array(subpack_values, dtype=np.int64)
        if packid==0:
            #sum
            packvalue = np.sum(values_np)
        elif packid==1:
            packvalue = np.product(values_np)
        elif packid==2:
            packvalue = np.min(values_np)
        elif packid==3:
            packvalue = np.max(values_np)
        elif packid==5:
            #greater than
            packvalue=0
            if values_np[0] > values_np[1]:
                packvalue=1
        elif packid==6:
            #less than
            assert values_np.size ==2 , "Error in values_np.size not 2"
            packvalue=0
            if values_np[0] < values_np[1]:
                packvalue=1
        elif packid==7:
            #equal than
            assert values_np.size ==2 , "Error in values_np.size not 2"
            packvalue=0
            if values_np[0] == values_np[1]:
                packvalue=1
                
    print(f"packvalue = {packvalue}")
    print(f"remaining_str = {remaining_str}")
    return {"value": packvalue, "remainstr": remaining_str}

In [12]:
readPacket(oppacket_bin) #OK

version: 1 , packid: 0
length type ID = 1
00000110101
n_subpackets=53
packet i=0
version: 4 , packid: 4
pack is a literal value
lit_value_str = 1001
lit_value_int=9
packvalue = 9
remaining_str = 0000011000000000100001010000000001000011111100111101111111001100110100011010011111101011111110101100000001001010001011001011100000000001100011000000010111111001001100000001011010001010000000000101010010000000100000000001011010000000010010101001000100000001000101000110001000000000010100001000000000011000000000000011011010111001100000000001011000100000000001010011100000000001001011000000001000111000000000000000011110001010101000000000011110000000000001010000010011100000000001001010100000000001100000100000000001011100100111100011001000010011001111101110110000000000001010000101100100000000001000010011000110011010000011101100001001010010000000000100001101100011001011000100111110000011101001100000000011111100011001111000100101010001010011001000000000101000011110011101000010111001011100111010100100100

{'value': 2223947372407, 'remainstr': '000'}

In [ ]:
#Output value 2219652405111, but it is wrong answer
#new answer changing dtype to int64, 2223947372407